# Model + RandomVariableクラスのコードリーディング

## setup

import

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import threading

from tensorflow_probability import edward2 as ed

/Users/yuki/.pyenv/versions/miniconda3-4.1.11/envs/ml_env/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/yuki/.pyenv/versions/miniconda3-4.1.11/envs/ml_env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Contextクラス

In [2]:
class Context(object):
    """Functionality for objects that put themselves in a context using
    the `with` statement.
    """
    contexts = threading.local()

    def __enter__(self):
        type(self).get_contexts().append(self)
        return self

    def __exit__(self, typ, value, traceback):
        type(self).get_contexts().pop()

withparentメソッド

In [3]:
def withparent(meth):
    """Helper wrapper that passes calls to parent's instance"""
    def wrapped(self, *args, **kwargs):
        res = meth(self, *args, **kwargs)
        if getattr(self, 'parent', None) is not None:
            getattr(self.parent, meth.__name__)(*args, **kwargs)
        return res
    # Unfortunately functools wrapper fails
    # when decorating built-in methods so we
    # need to fix that improper behaviour
    wrapped.__name__ = meth.__name__
    return wrapped

treedictクラス

In [4]:
class treedict(dict):
    """A dict that passes mutable extending operations used in Model
    to parent dict instance.
    Extending treedict you will also extend its parent
    """
    def __init__(self, iterable=(), parent=None, **kwargs):
        super(treedict, self).__init__(iterable, **kwargs)
        assert isinstance(parent, dict) or parent is None
        self.parent = parent
        if self.parent is not None:
            self.parent.update(self)
    # typechecking here works bad
    __setitem__ = withparent(dict.__setitem__)
    update = withparent(dict.update)

    def tree_contains(self, item):
        # needed for `add_random_variable` method
        if isinstance(self.parent, treedict):
            return (dict.__contains__(self, item) or
                    self.parent.tree_contains(item))
        elif isinstance(self.parent, dict):
            return (dict.__contains__(self, item) or
                    self.parent.__contains__(item))
        else:
            return dict.__contains__(self, item)


Modelクラス

In [8]:
class WithTreeModel(Context):
    def __new__(cls, *args, **kwargs):
        instance = super(WithTreeModel, cls).__new__(cls)
        if kwargs.get('model') is not None:
            instance.parent = kwargs.get('model')
        elif cls.get_contexts():
            instance.parent = cls.get_contexts()[-1]
        else:
            instance.parent = None
        return instance
            
    
    def __init__(self, name="", model=None, ):
        self.name = name
        if self.parent is not None:
            self.named_vars = treedict(parent=self.parent.named_vars)
        else:
            self.named_vars = treedict()
            
    @property
    def model(self):
        return self
    
    @property
    def decription(self):
        return
    
    @classmethod
    def get_contexts(cls):
        # no race-condition here, cls.contexts is a thread-local object
        # be sure not to override contexts in a subclass however!
        if not hasattr(cls.contexts, 'stack'):
            cls.contexts.stack = []
        return cls.contexts.stack

    @classmethod
    def get_context(cls):
        """Return the deepest context on the stack."""
        try:
            return cls.get_contexts()[-1]
        except IndexError:
            raise TypeError("No context on context stack")
            
    def add_random_variable(self, var):
        """Add a random variable to the named variables of the model."""
        if self.named_vars.tree_contains(var.name):
            raise ValueError(
                "Variable name {} already exists.".format(var.name))
        self.named_vars[var.name] = var

In [9]:
class Model(Context):
    def __new__(cls, **kwargs):
        instance = super(Model, cls).__new__(cls)
        if kwargs.get('model') is not None:
            instance.parent = kwargs.get('model')
        elif cls.get_contexts():
            instance.parent = cls.get_contexts()[-1]
        else:
            instance.parent = None
        return instance

    def __init__(self, name="", model=None, ):
        self.name = name
        self.named_vars = {}
        self.parent = model

    @property
    def model(self):
        return self

    @property
    def decription(self):
        return

    @classmethod
    def get_contexts(cls):
        # no race-condition here, cls.contexts is a thread-local object
        # be sure not to override contexts in a subclass however!
        if not hasattr(cls.contexts, 'stack'):
            cls.contexts.stack = []
        return cls.contexts.stack

    @classmethod
    def get_context(cls):
        """Return the deepest context on the stack."""
        try:
            return cls.get_contexts()[-1]
        except IndexError:
            raise TypeError("No context on context stack")

    def add_random_variable(self, var):
        """Add a random variable to the named variables of the model."""
        if var.name in self.named_vars:
            raise ValueError(
                "Variable name {} already exists.".format(var.name))
        self.named_vars[var.name] = var

RandomVariableクラス

In [10]:
class WithTreeRandomVariable(): # TODO: temporarily remove ed.RandomVariable
    
    def __init__(
                self,
#                 distribution,
#                 sample_shape=(),
                value=None,
                name="RV"
                ):
        self.model = WithTreeModel.get_context()
        self.name = name

#         super(RandomVariable, self).__init__(
#                                                 distribution,
#                                                 sample_shape,
#                                                 value,
#                                                 )
        
        self.model.add_random_variable(self)

In [25]:
class RandomVariable(): # TODO: temporarily remove ed.RandomVariable

    def __init__(
            self,
            name,
#             distribution,
#             sample_shape=(),
            value=None,
    ):
        self.context_stack = Model.get_contexts()
        self.model = Model.get_context()
        self.name = name

#         super(RandomVariable, self).__init__(
#             distribution,
#             sample_shape,
#             value,
#         )

        for model in self.context_stack:
            model.add_random_variable(self)

# 動作確認

【before】treedictクラスに依存している場合、ModelがネストするとparentにもRandomVariableが伝搬する

In [30]:
with WithTreeModel(name="model1") as model1:
    rv1 = WithTreeRandomVariable(name="rv1")
    with WithTreeModel(name="model2") as model2:
        rv2 = WithTreeRandomVariable(name="rv2")
        rv3 = WithTreeRandomVariable(name="rv3")
        print("model1: {}".format([v for v in model1.named_vars]))
        print("model2: {}".format([v for v in model2.named_vars]))

model1: ['rv1', 'rv2', 'rv3']
model2: ['rv2', 'rv3']


【after】treedictクラスに依存せず、RandomVarible内でModel.get_contextsを辿っている場合、ModelがネストするとparentにもRandomVariableが伝搬する

In [31]:
with Model(name="model1") as model1:
    rv1 = RandomVariable(name="rv1")
    with Model(name="model2") as model2:
        rv2 = RandomVariable(name="rv2")
        rv3 = RandomVariable(name="rv3")
        print("model1: {}".format([v for v in model1.named_vars]))
        print("model2: {}".format([v for v in model2.named_vars]))

model1: ['rv1', 'rv2', 'rv3']
model2: ['rv2', 'rv3']
